# 🔐 Seguridad, Compliance y Auditoría de Datos

Objetivo: implementar controles de seguridad (IAM, cifrado, enmascaramiento), cumplir regulaciones (GDPR, HIPAA, SOC2) y establecer auditoría de accesos y linaje.

- Duración: 120–150 min
- Dificultad: Alta
- Prerrequisitos: Governance (Senior 01), experiencia con cloud IAM

## 1. IAM y principio de mínimo privilegio

- Roles por función: data-engineer-ro, data-scientist, admin.
- Políticas granulares: lectura de bucket específico, escritura en tabla específica.
- MFA obligatorio para operaciones sensibles (producción, eliminación).
- Rotación automática de credenciales (secretos, keys).

In [ ]:
iam_policy_example = r'''
# AWS IAM policy para data engineer con acceso de lectura a raw y escritura a curated
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": ["s3:GetObject", "s3:ListBucket"],
      "Resource": ["arn:aws:s3:::data-lake/raw/*"]
    },
    {
      "Effect": "Allow",
      "Action": ["s3:PutObject", "s3:DeleteObject"],
      "Resource": ["arn:aws:s3:::data-lake/curated/*"]
    }
  ]
}
'''
print(iam_policy_example.splitlines()[:18])

## 2. Cifrado en tránsito y at-rest

- **At-rest**: S3 SSE-KMS, RDS encryption, disk encryption (EBS/GCS).
- **In-transit**: TLS 1.2+ para todas las APIs, VPN/PrivateLink para conectividad interna.
- Gestión de claves: AWS KMS, GCP Cloud KMS, Azure Key Vault con rotación automática.

## 3. Enmascaramiento y anonimización de PII

In [ ]:
import hashlib
def mask_email(email: str) -> str:
    user, domain = email.split('@')
    return f'{user[0]}***@{domain}'

def hash_pii(value: str) -> str:
    return hashlib.sha256(value.encode()).hexdigest()

mask_email('usuario@ejemplo.com'), hash_pii('12345678A')

## 4. Cumplimiento normativo

### GDPR (Europa)
- Derecho al olvido: implementar DELETE cascada y purga en backups.
- Consentimiento explícito y auditable.
- Data residency: almacenar en región EU.

### HIPAA (salud, USA)
- PHI cifrado, logs de acceso auditables.
- Business Associate Agreements con proveedores cloud.

### SOC2 (seguridad organizacional)
- Controles de acceso, monitoreo, incident response.
- Auditorías anuales por terceros.

## 5. Auditoría de accesos y linaje

- CloudTrail (AWS), Cloud Audit Logs (GCP), Activity Log (Azure).
- Registrar quién accedió qué dato, cuándo, desde dónde.
- Linaje de datos: OpenLineage, DataHub, Marquez → rastrear transformaciones y uso.
- Alertas ante accesos anómalos (SIEM: Splunk, Datadog Security).

## 6. Checklist de seguridad para pipelines

In [ ]:
checklist = '''
☑ IAM con mínimo privilegio y MFA
☑ Cifrado at-rest (KMS) y in-transit (TLS)
☑ Enmascaramiento de PII en logs y datasets de dev
☑ Rotación automática de secretos (API keys, DB passwords)
☑ Auditoría habilitada (CloudTrail, logs centralizados)
☑ Vulnerability scanning de contenedores (Trivy, Clair)
☑ Network segmentation (VPCs, subnets privadas)
☑ Incident response plan documentado y probado
'''
print(checklist)